# Gene biomarker selection using RFE
### Author: Shehbeel Arif
### Purpose: To find a set of genes most predictive of frailty based on the muscle gene expression data.

---

## Load libraries

In [10]:
# Library for data handling
import numpy as np
import pandas as pd

# Library for performing RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
#from sklearn.feature_selection import RFE

# Import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
# Import Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

# Import sklearn library to assess model accuracy
from sklearn.model_selection import cross_val_score

# Library for visualizations
import matplotlib.pyplot as plt
import seaborn as sns


## Data Preprocessing

In [4]:
# Load data
data_dir = '/Users/shehbeel/Documents/frailty-clinical-model/data/'
meta = pd.read_csv(data_dir + 'GSE144304_meta.txt', delimiter='\t')
counts = pd.read_csv(data_dir + 'GSE144304_raw_counts.csv', index_col='ENSEMBL_ID')

# Transpose counts
counts = counts.T
counts = counts.reset_index()

# Merge counts and meta data
df = pd.merge(meta, counts, left_on='sample_name', right_on='index').drop(['index'], axis=1)

# Sanity check
df

,sample_name,gender,treatment,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,...,ENSG00000284592.1,ENSG00000284594.1,ENSG00000284595.1,ENSG00000284596.1,ENSG00000284600.1,__no_feature,__ambiguous,__too_low_aQual,__not_aligned,__alignment_not_unique
0,s301,male,frail,99,20,284,101,19,15,252,...,0,0,0,0,0,1296261,302945,0,0,2856908
1,s302,male,frail,54,3,381,127,7,39,113,...,0,0,0,0,0,1302608,327263,0,0,3446180
2,s303,male,frail,44,2,303,123,9,16,147,...,0,0,0,0,0,1366217,301250,0,0,3418237
3,s304,male,frail,59,3,306,146,1,6,92,...,0,0,0,0,0,1540770,345828,0,0,3559093
4,s305,male,frail,52,0,402,103,10,31,115,...,0,0,0,0,0,1309163,337983,0,0,3084020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,s530,female,young,54,0,349,139,5,4,51,...,0,0,0,0,0,1110465,272000,0,0,2761633
76,s531,male,young,73,0,395,115,16,13,103,...,0,0,0,0,0,1103164,300310,0,0,3176856
77,s401,male,fit,55,2,252,97,4,17,48,...,0,0,0,0,0,1336720,268400,0,0,2459709
78,s404,male,fit,72,4,204,99,21,31,196,...,0,0,0,0,0,1363549,276365,0,0,2695024


In [8]:
# Split the dataset into training data and labels
X = df.iloc[:,3:].values
y = df.loc[:, df.columns == 'treatment'].values.ravel()

# Save feature names
columns = df.columns.to_list()
features = columns[3:]

# Sanity check
print(X[:5])
print(y[:5])

[[     99      20     284 ...       0       0 2856908]
 [     54       3     381 ...       0       0 3446180]
 [     44       2     303 ...       0       0 3418237]
 [     59       3     306 ...       0       0 3559093]
 [     52       0     402 ...       0       0 3084020]]
['frail' 'frail' 'frail' 'frail' 'frail']


---

## Automated Feature Selection using RFE-CV

In [11]:
rfecv = RFECV(estimator=RandomForestClassifier())
model = RandomForestClassifier()

pipeline = Pipeline([('Feature Selection', rfecv), ('Model', model)])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=123)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
np.mean(n_scores)

In [ ]:
# Run RFE-CV pipeline
pipeline.fit(X, y)

In [ ]:
print('Optimal number of features : %d' % rfecv.n_features_)

In [ ]:
#rfecv.support_
rfecv_df = pd.DataFrame(rfecv.ranking_,index=features,columns=['Rank']).sort_values(by='Rank',ascending=True)

rfecv_df.head()

In [ ]:
# Visualize the recursive feature elimination process
plt.figure(figsize=(12,6))
plt.xlabel('Number of features selected')
plt.ylabel('Cross validation score (nb of correct classifications)')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv_rank_df = pd.DataFrame(rfecv.ranking_, index=features, columns=['Rank']).sort_values(by='Rank', ascending=True)
rfecv_rank_df.head()
#rfecv_rank_df.to_csv('pbta_rfecv_selected_features.csv')

In [ ]:
rfecv_cols = ['Sample_ID', 'class'] + rfecv_rank_df[rfecv_rank_df['Rank']==1].index.to_list()
df[rfecv_cols]
#df[rfecv_cols].to_csv('pbta_rfecv_selected_features_df.csv')

---